<center>
    <h1> INF477 - Redes Neuronales Artificiales  </h1>
    <h2> Tarea #2 - Parte I: Entrenamiento de Autoencoders (AEs) y RBMs en MNIST </h2> 
</center>

Tarea realizada por:
* Alvaro Salinas - `alvaro.salinase@gmail.com` - **rol:** 201073001-8
* Martín Villanueva - `martin.villanueva@alumnos.usm.cl` - **rol:** 201104012-0

_DI UTFSM. Septiembre 2016._

## Tabla de Contenidos
* [1.0 Helper](#helper)
    * [Parte (a)](#1.0a)
* [1.1 Reducción de dimensionalidad](#dim)
    * [Parte (a)](#1.1a)
    * [Parte (b)](#1.1b)
    * [Parte (c)](#1.1c)
    * [Parte (d)](#1.1d)
    * [Parte (e)](#1.1e)
    * [Parte (f)](#1.1f)
    * [Parte (g)](#1.1g)
    * [Parte (h)](#1.1h)
    * [Parte (i)](#1.1i)
    * [Parte (j)](#1.1j)
* [1.2 Denoising](#den)
    * [Parte (a)](#1.2a)
    * [Parte (b)](#1.2b)
    * [Parte (c)](#1.2c)
    * [Parte (d)](#1.2d)
    * [Parte (e)](#1.2e)
    * [Parte (f)](#1.2f)
    * [Parte (g)](#1.2g)
* [1.3 Pre-entrenamiento](#pretr)
    * [Parte (a)](#1.3a)
    * [Parte (b)](#1.3b)
    * [Parte (c)](#1.3c)
    * [Parte (d)](#1.3d)
    * [Parte (e)](#1.3e)
    * [Parte (f)](#1.3f)
    * [Parte (g)](#1.3g)
    * [Parte (h)](#1.3h)

In [1]:
import pickle
import numpy as np

from keras.datasets import mnist
from keras.utils import np_utils
from keras.layers import Input, Dense
from keras.models import Model, load_model
from keras.optimizers import SGD

from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.neural_network import BernoulliRBM

Using Theano backend.


<div id='helper'/>
## 1.0 - Helper

<div id='1.0a'/>
### Parte (a)

In [13]:
def load_helper(nval=1000):
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    # 0-1 scaling
    X_train = X_train.astype('float32') / 255.
    X_test = X_test.astype('float32') / 255.
    # reshaping
    X_train = X_train.reshape((len(X_train), np.prod(X_train.shape[1:])))
    X_test = X_test.reshape((len(X_test), np.prod(X_test.shape[1:])))
    # training / validation split
    X_val = X_train[-nval:]
    y_val = y_train[-nval:]
    X_train = X_train[:-nval]
    y_train = y_train[:-nval]
    # correct format
    y_train = np_utils.to_categorical(y_train, 10)
    y_val = np_utils.to_categorical(y_val, 10)
    y_test = np_utils.to_categorical(y_test, 10)
    return X_train, X_val, X_test, y_train, y_val, y_test    

In [14]:
X_train, X_val, X_test, y_train, y_val, y_test = load_helper()

## 1.1 - Reducción de Dimensionalidad

### Parte (a)

In [8]:
input_img = Input(shape=(784,))
encoded = Dense(32, activation='sigmoid')(input_img)
decoded = Dense(784, activation='sigmoid')(encoded)
autoencoder = Model(input=input_img, output=decoded)
encoder = Model(input=input_img, output=encoded)
encoded_input = Input(shape=(32,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(input=encoded_input, output=decoder_layer(encoded_input))
autoencoder.compile(optimizer=SGD(lr=1.0), loss='binary_crossentropy')
autoencoder.fit(X_train, X_train, nb_epoch=50, batch_size=25, shuffle=True, validation_data=(X_val, X_val))
autoencoder.save('basic_autoencoder_768x32.h5')

Train on 59000 samples, validate on 1000 samples
Epoch 1/50
59000/59000 [==============================] - 8s - loss: 0.2943 - val_loss: 0.2756
Epoch 2/50
59000/59000 [==============================] - 8s - loss: 0.2651 - val_loss: 0.2735
Epoch 3/50
59000/59000 [==============================] - 7s - loss: 0.2640 - val_loss: 0.2732
Epoch 4/50
59000/59000 [==============================] - 8s - loss: 0.2635 - val_loss: 0.2727
Epoch 5/50
59000/59000 [==============================] - 7s - loss: 0.2633 - val_loss: 0.2728
Epoch 6/50
59000/59000 [==============================] - 8s - loss: 0.2631 - val_loss: 0.2721
Epoch 7/50
59000/59000 [==============================] - 8s - loss: 0.2597 - val_loss: 0.2661
Epoch 8/50
59000/59000 [==============================] - 8s - loss: 0.2543 - val_loss: 0.2615
Epoch 9/50
59000/59000 [==============================] - 9s - loss: 0.2503 - val_loss: 0.2577
Epoch 10/50
59000/59000 [==============================] - 8s - loss: 0.2463 - val_loss: 0.2530


### Parte (b)

In [ ]:
autoencoder = load_model('basic_autoencoder_768x32.h5')
#load other stuff ...
encoded_test = encoder.predict(X_test)
decoded_test = decoder.predict(encoded_test)
import matplotlib
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(X_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

### Parte (c)

In [ ]:
encoded_train = encoder.predict(x_train)
encoded_test = encoder.predict(x_test)
clf = KNeighborsClassifier(10)
clf.fit(encoded_train, y_train)
clf.fit(encoded_train, y_train)
score = clf.score(encoded_test,y_test)
print 'Classification Accuracy %.2f' % score

### Parte (d)

In [13]:
def clustering_accuracy(pred_labels,y,nclusters=10):
    true_pred = 0.0
    for i in range(0,nclusters):
        mvlabel = np.argmax(np.bincount(y[pred_labels==i]))
        true_pred += sum(y[pred_labels==i] == mvlabel)
    return true_pred/len(y)

In [ ]:
model = KMeans(n_clusters=10)
labels_pred = model.fit_predict(encoded_train)
score = metrics.adjusted_rand_score(y_train, labels_pred)
print 'Clustering ARI %.2f' % score
print 'Clustering ACC %.2f' % clustering_accuracy(labels_pred,y_train)

### Parte (e)

In [ ]:
pca = PCA(n_components=32)
pca.fit(x_train)
pca_train = pca.transform(x_train)
pca_test = pca.transform(x_test)
clf = KNeighborsClassifier(10)
clf.fit(pca_train, y_train)
score = clf.score(pca_test,y_test)
print 'PCA SCORE %.2f' % score

### Parte (f)

In [ ]:
model = BernoulliRBM(n_components=32, batch_size=25, learning_rate=0.05,verbose=1, n_iter=50) ##n_components is d'
model.fit(X_train) ##Train using persistent Gibbs chains
fileo = open('basicRBM.pickle','wb')
pickle.dump(model,fileo)
fileo.close()

### Parte (g)

In [18]:
target_dim = 2 #try other and do a nice plot
input_img = Input(shape=(784,))
encoded1 = Dense(1000, activation='relu')(input_img)
encoded2 = Dense(500, activation='relu')(encoded1)
encoded3 = Dense(250, activation='relu')(encoded2)
encoded4 = Dense(target_dim, activation='relu')(encoded3)
decoded4 = Dense(250, activation='relu')(encoded4)
decoded3 = Dense(500, activation='relu')(encoded3)
decoded2 = Dense(1000, activation='relu')(decoded3)
decoded1 = Dense(784, activation='sigmoid')(decoded2)
autoencoder = Model(input=input_img, output=decoded1)
encoder = Model(input=input_img, output=encoded3)
autoencoder.compile(optimizer=SGD(lr=1.0), loss='binary_crossentropy')
autoencoder.fit(X_train, X_train,nb_epoch=50,batch_size=25,shuffle=True, validation_data=(X_val, X_val))

Train on 59000 samples, validate on 1000 samples
Epoch 1/50
 4525/59000 [=>............................] - ETA: 45s - loss: 0.3053

KeyboardInterrupt: 

In [44]:
ls = [500, 250, 2]

In [45]:
ls[-2::-1]

[250, 500]

In [49]:
def train_deep_ae_helper(X_train, X_val, layer_sizes=[1000, 500, 250, 2], activation='relu'):
    layers_list = []
    layers_list.append( Input(shape=(784,)) )
    # encoding model
    for size in layer_sizes:
        layers_list.append( Dense(size, activation=activation)(layers_list[-1]) )
    # decoding model
    for size in layer_sizes[-2::-1]:
        layers_list.append( Dense(size, activation=activation)(layers_list[-1]) )
    layers_list.append( Dense(784, activation=activation)(layers_list[-1]) )
    # autoencoder model
    autoencoder = Model(input=layers_list[0], output=layers_list[-1])
    # training step
    autoencoder.compile(optimizer=SGD(lr=1.0), loss='binary_crossentropy')
    autoencoder.fit(X_train, X_train, nb_epoch=50, batch_size=25, shuffle=True, validation_data=(X_val, X_val))
    # storing step
    autoencoder.save('ae_{0}L_{1}d.h5'.format(len(layer_sizes), layer_sizes[-1]))
    return autoencoder

#### `Layer size: 2`

In [50]:
ae_2L_2d = train_deep_ae_helper(X_train, X_val, layer_sizes=[250, 2])

Train on 59000 samples, validate on 1000 samples
Epoch 1/50
59000/59000 [==============================] - 11s - loss: 4.5337 - val_loss: 5.1701
Epoch 2/50
43250/59000 [====================>.........] - ETA: 2s - loss: 5.0947

KeyboardInterrupt: 

In [ ]:
ae_2L_4d = train_deep_ae_helper(X_train, X_val, layer_sizes=[250, 4])

In [ ]:
ae_2L_8d = train_deep_ae_helper(X_train, X_val, layer_sizes=[250, 8])

In [ ]:
ae_2L_16d = train_deep_ae_helper(X_train, X_val, layer_sizes=[250, 16])

In [ ]:
ae_2L_32d = train_deep_ae_helper(X_train, X_val, layer_sizes=[250, 32])

#### `Layer size: 3`

In [ ]:
ae_3L_2d = train_deep_ae_helper(X_train, X_val, layer_sizes=[500, 250, 2])

In [ ]:
ae_3L_4d = train_deep_ae_helper(X_train, X_val, layer_sizes=[500, 250, 4])

In [ ]:
ae_3L_8d = train_deep_ae_helper(X_train, X_val, layer_sizes=[500, 250, 8])

In [ ]:
ae_3L_16d = train_deep_ae_helper(X_train, X_val, layer_sizes=[500, 250, 16])

In [ ]:
ae_3L_32d = train_deep_ae_helper(X_train, X_val, layer_sizes=[500, 250, 32])

#### `Layer size: 4`

In [ ]:
ae_4L_2d = train_deep_ae_helper(X_train, X_val, layer_sizes=[1000, 500, 250, 2])

In [ ]:
ae_4L_4d = train_deep_ae_helper(X_train, X_val, layer_sizes=[1000, 500, 250, 4])

In [ ]:
ae_4L_8d = train_deep_ae_helper(X_train, X_val, layer_sizes=[1000, 500, 250, 8])

In [ ]:
ae_4L_16d = train_deep_ae_helper(X_train, X_val, layer_sizes=[1000, 500, 250, 16])

In [ ]:
ae_4L_32d = train_deep_ae_helper(X_train, X_val, layer_sizes=[1000, 500, 250, 32])

### Parte (h)

### Parte (i)

### Parte (j)

## 1.1 - Reducción de dimensionalidad